In [1]:
# ライブラリ読み込み
import pandas as pd
import pronto

In [2]:
path_1 = "./UMLS2021AB/"
file_1="hp.obo"
#file_2 = "phenotype.hpoa"
#file_3 = "CTD_diseases.tsv.gz"
file_4 = "MRFILES.RRF.gz"
file_5 = "MRSTY.RRF.gz"
file_6 = "MRCONSO.RRF.aa.gz"
file_7 = "MRCONSO.RRF.ab.gz"
file_8 = "SRFIL"
file_9 = "SRDEF"

path_1 = "./UMLS2021AB/"
file_1="hp.obo"
file_2 = "phenotype.hpoa"
file_3 = "CTD_diseases.tsv.gz"
file_4 = "MRFILES.RRF.gz"
file_5 = "MRSTY.RRF.gz"
file_6 = "MRCONSO.RRF.aa.gz"
file_7 = "MRCONSO.RRF.ab.gz"
file_8 = "SRFIL"
file_9 = "SRDEF"

In [3]:
ont = pronto.Ontology.from_obo_library(file_1)
len(ont)

/home/admin_user/anaconda3/lib/python3.8/site-packages/pronto/ontology.py:206: UnicodeWarning: unsound encoding, assuming ISO-8859-1 (73% confidence)
  return cls(


16908

In [4]:
#HPOオントロジー中でXREFにUMLSを持つもの
#UMLSへのマッピングの数

hp=[]
umls = []
hp_umls=[]
hp_cui = []

for term in ont.terms():
    for each_xref in term.xrefs:
        if ("Xref('UMLS:" in str(each_xref)):
            cui = str(each_xref).replace("Xref('UMLS:", "").replace("')", "")
            hp.append(term.id) #HPO_id
            umls.append( cui) #CUI
            hp_umls.append([term.id, cui]) #HPO_id, CUI
            hp_cui.append([term.name + '(' + term.id + ')', term.id, cui]) #HPO_id, HPO_name, CUI
                
hp = sorted(list(set(hp)))
umls = sorted(list(set(umls)))
hp_umls = sorted(list(map(list, set(map(tuple, hp_umls)))))
hp_cui = sorted(list(map(list, set(map(tuple, hp_cui)))))

print("HPの数:", len(hp), "CUIの数:", len(umls), "マッピングの数:", len(hp_umls) ,len(hp_cui) )


HPの数: 11589 CUIの数: 12859 マッピングの数: 13005 13005


In [5]:
#2021 セマンティックタイプから,T184-CUI-HPOを調査

In [6]:
#"MRFILES.RRF.gz"
print(file_4)
mrfiles = pd.read_csv(path_1+file_4, sep="\|", header=None, engine="python")
mrfiles.columns=["FIL", "DES", "FMT", "CLS", "RWS" ,"BTS", 0]
mrfiles = mrfiles.iloc[:,:-1]

#FILがファイル名, FMTが列名
mrfiles["FMT_split"] = mrfiles ["FMT"].str.split(",")

MRFILES.RRF.gz


In [7]:
#列名取得
conso_col = mrfiles[mrfiles ["FIL"] =="MRCONSO.RRF"]["FMT_split"].values.tolist()[0] #2MRCONSO.RRF の列名
mrsty_col = mrfiles[mrfiles ["FIL"] =="MRSTY.RRF"]["FMT_split"].values.tolist()[0] #3"MRSTY.RRF"の列名

In [8]:
#"MRSTY.RRF.gz" Semantic Types: セマンティックタイプ
print(file_5)
mrsty_org = pd.read_csv(path_1+file_5, sep="\|", header=None, engine="python")
mrsty_org.columns = mrsty_col + [0]
mrsty_org = mrsty_org.iloc[:,:len(mrsty_col)]
#データがあるのはCUI, TUI列のみだったので２列だけに絞る
mrsty_org = mrsty_org[["CUI", "TUI"]]
print(len(mrsty_org))

MRSTY.RRF.gz
4864162


In [9]:
#T184を持つCUI
t184_cui = mrsty_org[ mrsty_org["TUI"] == "T184"]["CUI"].values.tolist()
print("mrsty.rrf のうち T184_CUI:", len(t184_cui))

#UMLSのT184に紐づけられたCUI。HPOオントロジーではHPOとCUIが紐づけられているので、CUIを使ってT184とHPOがマッピングできる
#HPO_CUIのペアのうち、T184_CUIとペアになっているHPOを取り出す
t184_hp = sorted(list(set([ i[0] for i in hp_umls  if i[1] in t184_cui])))
print("T184_CUI 紐づけらるHPO:", len(t184_hp))

#subsetのHPOのうち T184にあたるHPOの件数
#print("phenotype_dfのHPOのうち T184にあたるHPOの件数:", len(set(phenotype_df["HP"].values.tolist()) & set(t184_hp)))

mrsty.rrf のうち T184_CUI: 14281
T184_CUI 紐づけらるHPO: 216


In [11]:
print(file_6)
mrconso_aa_org = pd.read_csv(path_1+file_6, sep="\|", header=None, engine="python")
mrconso_aa_org.columns= conso_col + [0]
mrconso_aa_org = mrconso_aa_org.iloc[:,:len(conso_col)]

MRCONSO.RRF.aa.gz


In [12]:
print(file_7)
mrconso_ab_org = pd.read_csv(path_1+file_7, sep="\|", header=None, engine="python")
mrconso_ab_org = mrconso_ab_org.reset_index() #必要
mrconso_ab_org.columns=conso_col + [0]
mrconso_ab_org = mrconso_ab_org.iloc[:,:len(conso_col)]

MRCONSO.RRF.ab.gz


In [13]:
#mrconso_aa_orgの最後の行のデータの続きが mrconso_ab_orgの一行目にある。
#データをくっつけて一行にする
data = [i for i in mrconso_aa_org[-1:].values.tolist()[0]] +  [i for i in mrconso_ab_org[:1].values.tolist()[0]]
#[[enu,i] for enu, i in enumerate(data)]

rewrite_data = data[:3] + [str(data[3])+ str(data[18])] + data[19:-3]
#print(len(rewrite_data), rewrite_data)
tmp_df = pd.DataFrame(data=rewrite_data).T
tmp_df.columns=conso_col

18 ['C1507708', 'ENG', 'S', 'L8533703', 'PF', 'S10618931', 'Y', 'A18343928', nan, '35548-7', nan, 'LNC', 'LC', '35548-7', 'Fusarium moniliforme IgG Ab [Mass/volume] in Serum', '0', 'N', nan]


In [14]:
#調整
mrconso_aa = mrconso_aa_org[:-1]
mrconso_ab = mrconso_ab_org.append(tmp_df)[1:].sort_index() #mrconso_ab_org.update(df)
print(len(mrconso_aa_org), len(mrconso_aa), len(mrconso_ab_org), len(mrconso_ab) )

8683998 8683997 7859674 7859674


In [15]:
#mrconso_aa, mrconso_ab 合体
conso = pd.concat([mrconso_aa, mrconso_ab ] )
conso = conso.reset_index(drop=True)
print(len(conso))#hpo["LAT"].unique() #ENGのみ

16543671


In [16]:
#conso.to_csv("conso_2021AB.csv", index=False)

In [91]:
conso[["CUI", "STR"]][:3]#STR=各ソースでのCUI名

,CUI,STR
0,C0000005,(131)I-Macroaggregated Albumin
1,C0000005,(131)I-MAA
2,C0000005,Macroagrégats d'albumine marquée à l'iode 131


In [76]:
#CUI, HPO, STR(ターム名）

#HPO ONTOLOGY の HPO-CUI
hp_umls_df = pd.DataFrame(hp_umls, columns=["HPO", "CUI"])
#CONSO の CUI-STR(各ソースでの名前) と合わせる
conso_hp_cui = pd.merge(hp_umls_df, conso[["CUI", "STR"]] ) #STR=各ソースでのCUI名
conso_hp_cui = conso_hp_cui[~conso_hp_cui.duplicated()]
len(conso_hp_cui)

273441

In [92]:
conso_hp_cui[:10]

,HPO,CUI,STR
0,HP:0000001,C0444868,All
9,HP:0000001,C0444868,ALL
10,HP:0000001,C0444868,all
12,HP:0000001,C0444868,All (qualifier value)
13,HP:0000001,C0444868,en todo
14,HP:0000001,C0444868,todo
15,HP:0000001,C0444868,todos
16,HP:0000001,C0444868,todos (calificador)
17,HP:0000002,C4025901,Abnormality of body height
19,HP:0000003,C3714581,multicystické dysplastické ledviny


In [87]:
mrsty_org[:3]

,CUI,TUI
0,C0000005,T116
1,C0000005,T121
2,C0000005,T130


In [77]:
#セマンティックデータ(CUI-TUI) と合わせる
#CUI-TUI-HPO-STR(各ソースでのCUI名）がつながる
semtypes = pd.merge(mrsty_org, conso_hp_cui ) 
print(len(semtypes), len(semtypes["CUI"].unique()), len(semtypes["TUI"].unique()), len(semtypes["HPO"].unique()))
semtypes[:5]

273774 12703 33 11475


,CUI,TUI,HPO,STR
0,C0000731,T033,HP:0003270,ABDOMENAREN BOLUMEN ALDAKETA/ABDOMEN HAUNDITZEA
1,C0000731,T033,HP:0003270,Břišní distenze
2,C0000731,T033,HP:0003270,Vzedmuté břicho
3,C0000731,T033,HP:0003270,Zduřené břicho
4,C0000731,T033,HP:0003270,AEndring af abdom omfang/udspiling


In [78]:
#HPO名を割り当てる
semtypes["HPO_name"] = [ont[i].name for i in semtypes["HPO"]]
semtypes["HPO_id_name"] = [ont[i].name + '(' + i + ')' for i in semtypes["HPO"]]
semtypes["CUI_STR"] = [ i[0] + '(' + i[1] + ')' for i in semtypes[["CUI", "STR"]].values.tolist() ]
semtypes = semtypes[["HPO_id_name", "HPO", "CUI", "CUI_STR","TUI"]]
print(len(semtypes), semtypes["HPO"].nunique())

273774 11475


In [79]:
#TUI名を割り当てる
#https://lhncbc.nlm.nih.gov/semanticnetwork/SemanticNetworkArchive.html
#SRFILファイルから　SRDEFファイルの列名を取得

print(file_8)
srfil = pd.read_table(path_1+file_8, sep="\|", header=None, engine='python')
srfil["col_name"] = srfil [2].str.split(",")
srdef_col = srfil[srfil [0] =="SRDEF"]["col_name"].values.tolist()[0] 

print(file_9)
srdef = pd.read_table(path_1+file_9, sep="\|", header=None, engine='python')
srdef.columns= srdef_col+[0]
srdef = srdef.iloc[:, :-1]
srdef[ (srdef["RT"] == "STY") & (srdef["UI"] == "T184")]

tui_name = srdef[ (srdef["RT"] == "STY") ][["UI", "STY/RL"]]
tui_name = tui_name.rename(columns={"UI":"TUI", "STY/RL":"TUI_name"})


SRFIL
SRDEF


In [82]:
hpo_semtypes = pd.merge(semtypes, tui_name)
print(len(hpo_semtypes))
print([ [i, hpo_semtypes[i].nunique()] for i in hpo_semtypes.columns])
hpo_semtypes[:3]

273774
[['HPO_id_name', 11475], ['HPO', 11475], ['CUI', 12703], ['CUI_STR', 261309], ['TUI', 33], ['TUI_name', 33]]


,HPO_id_name,HPO,CUI,CUI_STR,TUI,TUI_name
0,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(ABDOMENAREN BOLUMEN ALDAKETA/ABDOMEN ...,T033,Finding
1,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(Břišní distenze),T033,Finding
2,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(Vzedmuté břicho),T033,Finding


In [83]:
print(semanitic_df[semanitic_df["TUI"]=="T184"]["HPO"].nunique())
print(semanitic_df[semanitic_df["TUI"]=="T184"]["CUI"].nunique())

216
218


#集計
shukei_df = hpo_semtypes.groupby(["TUI", "TUI_name"]).agg({'CUI':list, "HPO":list})

shukei_df["CUI"] = [list(set(i)) for i in shukei_df["CUI"]]
shukei_df["HPO"] = [list(set(i)) for i in shukei_df["HPO"]]

shukei_df["CUI_num"] = [len(i) for i in shukei_df["CUI"]]
shukei_df["HPO_num"] = [len(i) for i in shukei_df["HPO"]]

shukei_df = shukei_df.reset_index()
shukei_df[shukei_df["TUI"]=="T184"][["TUI", "TUI_name", "CUI_num", "HPO_num"]]
cui_freq = shukei_df[["TUI", "TUI_name", "CUI_num"]].sort_values("CUI_num", ascending=False).reset_index(drop=True)
hpid_freq= shukei_df[["TUI", "TUI_name", "HPO_num"]].sort_values("HPO_num", ascending=False).reset_index(drop=True)